In [1]:
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = '16'
plt.rcParams['figure.dpi'] = 125
plt.rcParams['figure.facecolor'] = 'white'

import numpy as np
import struct, os
import time
import xarray as xr
import pandas as pd

import obspy
from obspy import read_inventory, read, signal, UTCDateTime, Stream, Trace


In [2]:
from src.pyoad.pyoad import read_data
header_df, waveforms = read_data('/Users/gilaverbuch/dev/SHRU_programs/SHRU_24bit/05140537.D21')

TypeError: read_waveforms_() missing 1 required positional argument: 'header_df'

In [ ]:
test = [[None]*2]*3
test

In [ ]:
test[0:1].append(2)

In [ ]:
test

In [ ]:
int(header_df.loc['channels'].values)

In [ ]:
waveforms.plot();

In [ ]:
waveforms[2].stats

In [ ]:
fig = waveforms[1].spectrogram(wlen=1, per_lap=0.5, log=True, dbscale=True, show=False);
fig.set_figheight(8)
fig.set_figwidth(15)
plt.show()

In [ ]:
tr = waveforms[3].copy()
tr.filter('bandpass', freqmin=10,freqmax=100,  corners=4, zerophase=True)
tr.plot();


In [ ]:
print(waveforms[0])
tr = waveforms[0].copy()
tr.decimate(factor=2, strict_length=False)

In [ ]:
tr = waveforms[0].copy()
tr.filter('bandpass', freqmin=1e1,freqmax=8e2,  corners=4, zerophase=True)

data_envelope = obspy.signal.filter.envelope(tr.data)

In [ ]:
data = waveforms[0].data
npts = waveforms[0].stats.npts
samprate = waveforms[0].stats.sampling_rate
t = np.arange(0, npts / samprate, 1 / samprate)

plt.figure()
plt.plot(t, waveforms[0].data, 'k')
plt.plot(t, data_envelope.data, 'r')
plt.title(waveforms[0].stats.starttime)
plt.ylabel('Filtered Data w/ Envelope')
plt.xlabel('Time [s]')
plt.xlim(11, 11.2)
plt.show()

In [ ]:
file_name = '/Users/gilaverbuch/dev/SHRU_programs/SHRU_24bit/05140537.D21'
matlab_channel = np.loadtxt('/Users/gilaverbuch/dev/SHRU_programs/SHRU_24bit/channel1.scv')
# Header = read_header()

In [ ]:
samp_freq = 19531.25
dt = 1/samp_freq
time_m = np.arange(0, len(matlab_channel)*dt, dt)

plt.figure(figsize = (15,5))
plt.plot(time_m, matlab_channel*1e-4, 'r')

plt.show()

# My python code

In [ ]:
f_data = open(file_name, "rb")  # reopen the file
x = f_data.read()

In [ ]:
xs = bytearray(b'\x00\x1b\xcf')
xs.append(0)
print(xs)
struct.unpack('>i', b'\x00\x1b\xcf\x00')[0], int.from_bytes(xs, byteorder='big', signed=True)

In [ ]:
pos = 1024
l = (len(x[1024:])//128)
npts = 1048576

l1 = l*0
l2 = l1+l
def get_data(x, pos, l):
    
    data=[]
    for loc in range(l1+pos,l2, 12):
    
        d = bytearray(x[loc:loc+3])
        d.append(0)
        dpoint = int.from_bytes(d, byteorder='big', signed=True) * (2.5/(2**23)/20)
        data.append(dpoint)
    
    return data
    
    

In [ ]:
start = time.time()

channel1 = get_data(x, pos, l)

end  = time.time()
print ("Time elapsed:", end - start)

In [ ]:
len(matlab_channel) / len(channel1)

In [ ]:
channel1 = np.asarray(channel1, dtype=np.float32)

In [ ]:
samp_freq = 19531.25
dt = 1/samp_freq
time_p = np.arange(0, len(channel1)*dt, dt)

plt.figure(figsize = (15,5))
plt.plot(time_p, channel1, 'g')
# plt.plot(time_p, channel2*2.5e2, 'r')
# plt.xlim(5.8,6)
plt.show()

In [ ]:
f, t, Sxx = signal.spectrogram(channel1, samp_freq, nperseg=int(samp_freq), noverlap=int(samp_freq//2))
plt.figure(figsize=(10,5))
plt.pcolormesh(t, f, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
# plt.xlim(10,20)
plt.ylim(0,200)
# plt.clim(-10, -5)
plt.colorbar()
plt.title('from python')
plt.show()

# Data from YT's matlab code

In [ ]:

f, t, Sxx = signal.spectrogram(matlab_channel, samp_freq, nperseg=int(samp_freq), noverlap=int(samp_freq//2))
plt.figure(figsize=(10,5))
plt.pcolormesh(t, f, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
# plt.xlim(10,20)
plt.ylim(0,200)
plt.clim(0,1e-9)
plt.colorbar()
plt.title('from matlab')
plt.show()